In [1]:
import pickle

In [2]:
import os
from tqdm.notebook import tqdm
from typing import Dict
from collections import defaultdict
import numpy as np

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

def numLines(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1
def loadData(filename, max_points):
    file_len = numLines(filename)
    f = open(filename, 'r')
    inputs = []
    outputs = []
    for i in tqdm(range(file_len)):
        if i == max_points:
            break
        line = f.readline()
        if line[-1] == '\n':
            line = line[:-1]
        line = line.split('\t')
        inputs.append(line[0])
        outputs.append(line[1])
    data = {'inputs': inputs, 'outputs': outputs}
    return data
        
def load_entity_strings(filename):
    with open(filename) as f:
        lines = f.read().splitlines()
    return lines

def get_entity_wd_id_dict(filename):
    out = {}
    f = open(filename, 'r')
    for line in f:
        if line[-1] == '\n':
            line = line[:-1]
        line = line.split('\t')
        out[line[1]] = line[0]
    return out
    

def create_filter_dict(data) -> Dict[str, int]:
    filter_dict = defaultdict(list)
    for input, output in tqdm(zip(data["inputs"], data["outputs"])):
        filter_dict[input].append(output)
    return filter_dict

def getAllFilteringEntities(input, filter_dicts):
    entities = []
    splits = ['train', 'test', 'valid']
    for s in splits:
        entities.extend(filter_dicts[s][input])
    return list(set(entities))

def wikidata_link_from_id(id):
    uri = 'https://www.wikidata.org/wiki/' + id
    return uri

In [3]:
# dataset_name = 'olpbench'
# entity_strings = load_entity_strings(os.path.join("data", dataset_name, "entity_strings.txt"))

In [4]:
# entity_strings_set = set(entity_strings)
# len(entity_strings_set)

In [5]:
data = {}
splits = ['train', 'valid', 'test']
dataset_name = 'olpbench'
for split in splits:
    data[split] = loadData(os.path.join('data', dataset_name, split + '.txt'), -1)

  0%|          | 0/61301566 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

In [6]:
filter_dicts = {}
splits = ['train', 'valid', 'test']
for split in splits:
    filter_dicts[split] = create_filter_dict(data[split])

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [87]:
# fname = 'scores.pickle'
# fname = 'scores/scores_full_codexm_small.pickle'
fname = 'scores/scores_a.pickle'
# fname = 'scores_500_base_trie.pickle'
scores_data = pickle.load(open(fname, 'rb'))

In [88]:
id = 2
print(scores_data['input_strings'][id], scores_data['target_strings'][id])
print(scores_data['prediction_strings'][id])

predict tail: london | has:impl_poss-clause zonal fare system used by | the london underground
the saskatchewan railway


In [97]:
from unidecode import unidecode
def load_test_data_olpbench(fname):
    # assumes linked data format
    # s r o s1|||s2... o1|||o2...
    f = open(fname)
    tuples = []
    for line in f:
        if line[-1] == '\n':
            line = line[:-1]
        line = line.split('\t')
        tuples.append(line)
    out = []
    for t in tqdm(tuples):
        for s_id, o_id in zip([3,4],[4,3]):
            rel = t[1]
            s = unidecode(t[s_id]).split('|||')
            o = unidecode(t[o_id]).split('|||')
            out.append((s, rel, o))
    return out

In [98]:
test_data = load_test_data_olpbench('data/olpbench/test_data.txt')

  0%|          | 0/10000 [00:00<?, ?it/s]

In [99]:
id = 117
print(scores_data['prediction_strings'][id])
test_data[id]

the station


(['the gjovik line'],
 'is located on',
 ['a train station',
  'a station',
  'the station',
  'a railway station',
  'train stations'])

In [100]:
len(scores_data['prediction_strings'])

20000

In [103]:
count = 0
for id in range(len(scores_data['prediction_strings'])):
    targets = test_data[id][2]
    predictions = scores_data['prediction_strings'][id]
    # if 1 predictions make array
    predictions = [predictions]
    if len(set(targets).intersection(set(predictions))) > 0:
        count += 1
        print(id)
total_count = len(scores_data['prediction_strings'])
count/total_count

8
76
117
202
206
232
238
255
264
284
334
402
406
636
751
786
800
852
860
863
925
926
988
1000
1022
1033
1044
1092
1114
1141
1159
1362
1434
1456
1460
1463
1478
1496
1544
1604
1615
1646
1721
1726
1730
1760
1774
1809
1815
1822
1828
1924
1960
1988
2000
2055
2140
2162
2183
2305
2364
2375
2376
2394
2536
2578
2610
2645
2735
2788
2818
2822
2874
2936
2939
3022
3048
3186
3215
3218
3227
3228
3254
3270
3282
3302
3384
3400
3480
3516
3560
3766
3776
3787
3826
3835
3857
3898
3916
3952
3966
3995
3996
4003
4027
4077
4145
4149
4175
4178
4204
4270
4310
4315
4342
4374
4417
4424
4438
4476
4505
4538
4580
4585
4612
4620
4663
4673
4732
4807
4810
4813
4824
4828
4882
4974
5083
5119
5124
5153
5223
5316
5336
5346
5348
5364
5387
5403
5430
5446
5454
5460
5464
5514
5536
5546
5593
5664
5725
5737
5784
5800
5813
5852
5858
5876
5912
5916
5950
6014
6041
6058
6129
6213
6216
6238
6294
6316
6329
6336
6345
6398
6450
6471
6478
6517
6545
6546
6554
6558
6596
6619
6654
6669
6678
6692
6718
6741
6747
6778
6820
6844
6860
6873
6890
6

0.02905

In [69]:
count

314

In [8]:
predictions_scores_dicts = []
for string_arr, score_arr in tqdm(zip(scores_data['prediction_strings'], scores_data['scores'])):
    ps_pairs = [(p,s) for p,s in zip(string_arr, score_arr)]
    ps_pairs = list(set(ps_pairs)) # while sampling, duplicates are created
    # remove predictions that are not entities
    ps_dict_only_entities = defaultdict(list)
    for ps in ps_pairs:
        if ps[0] in entity_strings_set:
            ps_dict_only_entities[ps[0]] = ps[1]
    predictions_scores_dicts.append(ps_dict_only_entities)

61244it [00:11, 5378.53it/s]


In [9]:
max([len(predictions_scores_dicts[x]) for x in range(500)])

377

In [10]:
id = 2
print(scores_data['input_strings'][id], scores_data['target_strings'][id])
# print(scores_data['scores'][2])
predictions_scores_dicts[id]

predict tail: vietnam | diplomatic relation | mexico


defaultdict(list,
            {'serbia': -3.3982773,
             'luxembourg': -4.66638,
             'indonesia': -3.5668106,
             'germany': -3.5101147,
             'mexico': -3.095539,
             'brazil': -5.24856,
             'italy': -3.5092916,
             'ukraine': -4.684395,
             'cambodia': -4.270482,
             'spain': -3.7536564,
             'saudi arabia': -3.193513,
             'soviet union': -5.5200253,
             'mongolia': -3.908885,
             'greece': -5.595957,
             'european union': -5.8609905,
             'denmark': -3.6262321,
             'laos': -3.848981,
             'bangladesh': -3.5050812,
             'philippines': -3.3096695,
             'poland': -5.6494665,
             'bulgaria': -3.183054,
             'australia': -3.7198315,
             'cuba': -4.132122,
             'singapore': -4.7629385,
             'vietnam': -4.123703,
             'turkey': -5.0184784,
             'georgia': -4.5141897,
    

In [11]:
len(predictions_scores_dicts)

61244

In [12]:
import numpy as np
predictions_filtered = []
head_num_filter = 0
tail_num_filter = 0
hits_at_all = 0
count = 0
for i in tqdm(range(len(predictions_scores_dicts))):
    ps_dict = predictions_scores_dicts[i].copy()
    target = scores_data['target_strings'][i]
    inputs = scores_data['input_strings'][i]
    prediction_strings = ps_dict.keys()
    if target in prediction_strings:
        original_score = ps_dict[target]
    # get filtering entities
    filtering_entities = getAllFilteringEntities(inputs, filter_dicts)
    if len(filtering_entities) == 1:
        count += 1
    if 'head' in inputs:
        head_num_filter += len(filtering_entities)
    else:
        tail_num_filter += len(filtering_entities)
    for ent in filtering_entities:
        if ent in ps_dict:
            ps_dict[ent] = -float("inf")
    if target in prediction_strings:
        ps_dict[target] = original_score
        hits_at_all += 1
    # softmax for scores
    names_arr = []
    scores_arr = []
    for k, v in ps_dict.items():
        names_arr.append(k)
        scores_arr.append(v)
    scores_arr = np.array(scores_arr)
#     scores_arr = softmax(scores_arr)
    for name, score in zip(names_arr, scores_arr):
        ps_dict[name] = score
    predictions_filtered.append(ps_dict)
print(head_num_filter/len(predictions_filtered), tail_num_filter/len(predictions_filtered))
print(hits_at_all/len(predictions_filtered))
print(count)

100%|████████████████████████████████████████████████████████████| 61244/61244 [00:29<00:00, 2083.73it/s]

1374.5308111815034 1.9256090392528247
0.36741558356736986
14383


In [13]:
count = {}
reciprocal_ranks = 0.0
k_list = [1,3,10]
for k in k_list:
    count[k] = 0
num_small_arrs = 0
count2 = 0
for i in range(len(predictions_filtered)):
    target = scores_data['target_strings'][i]
    ps_dict = predictions_filtered[i]
    ps_sorted = sorted(ps_dict.items(), key=lambda item: -item[1])
    if len(ps_dict) == 0:
        preds = []
    else:
        preds = [x[0] for x in ps_sorted]
    if target in preds:
        rank = preds.index(target) + 1
        reciprocal_ranks += 1./rank
    for k in k_list:
        if target in preds[:k]:
            count[k] += 1
    if len(preds) < 10 and target not in preds:
        num_small_arrs += 1
    if target in preds:
        count2 += 1
for k in k_list:
    hits_at_k = count[k]/len(predictions_filtered)
    print('hits@{}'.format(k), hits_at_k)
print('mrr', reciprocal_ranks/len(predictions_filtered))
print(num_small_arrs/len(predictions_filtered), 'were <10 length preds array without answer')

hits@1 0.2106002220625694
hits@3 0.28918751224609757
hits@10 0.3459440924825289
mrr 0.2571287581304741
0.08693096466592645 were <10 length preds array without answer


In [75]:
count2/len(predictions_filtered)

0.4732324701774804

In [75]:
id = 4
inputs = scores_data['input_strings'][id]
preds = predictions_filtered[id]
preds = sorted(preds.items(), key=lambda item: -item[1])
target = scores_data['target_strings'][id]

In [76]:
print(inputs, 'Target:', target)
preds[:10], target

predict tail: Valenciennes | isLocatedIn | Target: Nord-Pas-de-Calais


([('Île-de-France', 0.3138781066564107),
  ('Lower_Silesian_Voivodeship', 0.13227073610334242),
  ('Nord-Pas-de-Calais', 0.09675851055651631),
  ('Valenciennes', 0.07590361486184467),
  ('Lists_of_World_Heritage_Sites_in_Europe', 0.061758891746029325),
  ('Centre_region,_France', 0.052148043026076424),
  ('Lesser_Poland_Voivodeship', 0.044005159013402746),
  ('Pays_de_la_Loire', 0.04173769409652669),
  ('Paris', 0.03586650379301719),
  ('Quebec', 0.030035223333408627)],
 'Nord-Pas-de-Calais')

In [482]:
# only head/tails
count = 0
for id in range(60,120, 2):
    inputs = scores_data['input_strings'][id]
    preds = predictions_filtered[id]
    preds = sorted(preds.items(), key=lambda item: -item[1])
    target = scores_data['target_strings'][id]
    pred1 = preds[0][0]
    if pred1 == target:
        print(int(id/2), inputs, pred1)
        count += 1
'count', count

32 predict tail: ali kazemaini | birthplace | tehran
34 predict tail: ashta, maharashtra | instance of | human settlement
37 predict tail: roy shaw 0 | instance of | human being
40 predict tail: t. canby jones | has surname | jones (family name)
45 predict tail: naveen kumar | instance of | human being
46 predict tail: barlow respiratory hospital | host country | united states of america
48 predict tail: camiling | office held by head of government | mayor
51 predict tail: hazel soan | instance of | human being
52 predict tail: efrain herrera | sport played | association football
53 predict tail: oluf munck | instance of | human being
54 predict tail: thomas gilchrist | instance of | human being
58 predict tail: desmoplastic fibroma | subclass of | fibroma


('count', 12)

In [419]:
%%html
print("<a href='your_url_here'>Showing Text</a>")


In [364]:
e2wdid['pakistan']

'Q4121082'

In [174]:
sequences = ['english', 'english language', 'french']
t = Trie(sequences)

In [178]:
t.get('x')

[]